# GOAL

Anonymize data from AR app to allow public sharing.
- AR Comments (OK)
- Countries (N/A)
- DSO (OK)
- ExchangeRates (N/A)
- Invoice Item Detail
- Invoices
- Items
- Link Table
- Product Lines
- Subsidiaries (OK)

# PACKAGES

In [34]:
import pandas as pd
from anonympy.pandas import dfAnonymizer
from anonympy.pandas.utils_pandas import available_methods
from anonympy.pandas.utils_pandas import fake_methods
import os
import gcsfs
import pickle
from random import shuffle

# PARAMETERS

In [44]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../secrets/gcp_qlik_key.json'
source_path='gs://qlik-demos-data/finance/in/'
destination_path='gs://qlik-demos-data/finance/out/'
pd_options = {"token": os.environ['GOOGLE_APPLICATION_CREDENTIALS']}
fs = gcsfs.GCSFileSystem(token=os.environ['GOOGLE_APPLICATION_CREDENTIALS'])

# seeds and keys for anonymization
key = 'qlikrulesaboveallothers'
seed = 1001

# FUNCTIONS

In [45]:
def noise_amount_column(original_column):
    noise_column=original_column.replace(".-","-0.",regex=True).astype('float')
    return noise_column.apply(lambda x: round(x*2/3+50000,1) if x>=0 else round(x*2/3-50000,1))

In [106]:
def scramble_str(original_str):
    def return_number(number=0.3):
        return number
    
    scrambled_str=list(original_str)
    shuffle(scrambled_str,return_number)

    return "".join([str(item) for item in scrambled_str])

#scramble_str('123456789')
assert scramble_str("123456789")=="471562893"

/var/folders/qq/4mccktj53_scgd15__4s1jvh0000gn/T/ipykernel_46833/2055437299.py:6: DeprecationWarning: The *random* parameter to shuffle() has been deprecated
since Python 3.9 and will be removed in a subsequent version.
  shuffle(scrambled_str,return_number)


In [90]:
def scramble_column(original_column):
    scrambled_column=original_column.copy()
    return scrambled_column.apply(scramble_str)

In [151]:
def sequencial_values_for_column(original_column):
    columns_names={'index':'new',0:'original'}
    sequencial_values_for_column=pd.DataFrame(set(original_column)).reset_index().rename(columns=columns_names)
    sequencial_values_for_column=pd.merge(
        original_column,
        sequencial_values_for_column,
        left_on=original_column.name,
        right_on='original',
        how='left').drop(
            columns=['original',original_column.name])
    return sequencial_values_for_column.rename(columns={'new':original_column.name})
    #return original_column

# DATA ANONYMIZATION

## Subsidiaries

In [46]:
# read original file from gcs
df_subsidiaries=pd.read_csv(source_path+'AR_Subsidiaries.csv',storage_options=pd_options)
df_subsidiaries['NetSuite Subsidiary ID']=df_subsidiaries['NetSuite Subsidiary ID'].astype('str')

print('original dataframe')
display(df_subsidiaries.head())

# anonymize dataframe
anon_subsidiaries = dfAnonymizer(df_subsidiaries)

anon_subsidiaries.categorical_tokenization('%SubsidiaryCode',max_token_len=3,key=key)
anon_subsidiaries.categorical_fake({'Subsidiary':'company'},seed=seed)
anon_subsidiaries.column_suppression(['Is Attunity Subsidiary','VAT Registration Number'])
anon_subsidiaries.categorical_resampling(
    ['Subsidiary Currency Code','Subsidiary Region'],seed=seed)

print(anon_subsidiaries.info())

df_subsidiaries_anon=anon_subsidiaries.to_df()
df_subsidiaries_anon['NetSuite Subsidiary ID']=df_subsidiaries_anon['%SubsidiaryCode']
df_subsidiaries_anon['Subsidiary Legal Name']=df_subsidiaries_anon['Subsidiary']
df_subsidiaries_anon['Workday Subsidiary Name']=df_subsidiaries_anon['Subsidiary']
df_subsidiaries_anon['Subsidiary Region']=df_subsidiaries_anon['Subsidiary Region'].replace({'Technologies':'World'},inplace=False)

# merge original and anonymized dataframes
df_subsidiaries=df_subsidiaries.join(df_subsidiaries_anon,how='inner',lsuffix='_orig')
print('full dataframe')
display(df_subsidiaries.head())

# persist anonymized df to GCS
df_subsidiaries_anon.to_csv(destination_path+'anon_subsidiaries.csv',index=False)

# persist mapping tables to GCS
map_subsidiary_code = dict(zip(df_subsidiaries['%SubsidiaryCode_orig'], df_subsidiaries['%SubsidiaryCode']))
map_subsidiary_currency_code = dict(zip(df_subsidiaries['Subsidiary Currency Code_orig'], df_subsidiaries['Subsidiary Currency Code']))

with fs.open(destination_path+'map_subsidiary_code.pickle', 'wb') as handle:
    pickle.dump(map_subsidiary_code, handle, protocol=pickle.HIGHEST_PROTOCOL)
with fs.open(destination_path+'map_subsidiary_currency_code.pickle', 'wb') as handle:
    pickle.dump(map_subsidiary_currency_code, handle, protocol=pickle.HIGHEST_PROTOCOL)

df_subsidiaries = anon_subsidiaries=df_subsidiaries_anon=map_subsidiary_code=map_subsidiary_currency_code=[]



original dataframe


,Subsidiary,%SubsidiaryCode,Subsidiary Legal Name,Subsidiary Currency Code,Subsidiary Is Active,NetSuite Subsidiary ID,VAT Registration Number,Is Active,Is Elimination,Subsidiary Region,Workday Subsidiary Name,Is Attunity Subsidiary
0,Qlik Foreign Parent AB - Do Not Use,NA1,Qlik Foreign Parent AB - Do Not Use,SEK,No,54,NaN,No,No,NaN,NaN,NaN
1,Expressor Software Corporation,EXP,Expressor Software Corporation,USD,No,46,NaN,No,No,Americas,Expressor Software Corporation,No
2,Purchase Price Adjustments,PPA,Purchase Price Adjustments,USD,Yes,56,NaN,Yes,No,Technologies,NaN,No
3,QlikTech Holdings Inc.,HOI,QlikTech Holdings Inc.,USD,Yes,13,NaN,Yes,No,Technologies,NaN,Yes
4,QlikTech Belgium,BEL,QlikTech Belgium,EUR,No,5,BE0848691897,No,No,EMEA,NaN,No


+--------------------------+--------+-------------+--------------------+
|          Column          | Status |    Type     |       Method       |
+==========================+========+=============+====================+
| Subsidiary               | 1      | categorical | Synthetic Data     |
+--------------------------+--------+-------------+--------------------+
| %SubsidiaryCode          | 1      | categorical | Tokenization       |
+--------------------------+--------+-------------+--------------------+
| Subsidiary Legal Name    | 0      | categorical |                    |
+--------------------------+--------+-------------+--------------------+
| Subsidiary Currency Code | 1      | categorical | Resampling         |
+--------------------------+--------+-------------+--------------------+
| Subsidiary Is Active     | 0      | categorical |                    |
+--------------------------+--------+-------------+--------------------+
| NetSuite Subsidiary ID   | 0      | categorical |

,Subsidiary_orig,%SubsidiaryCode_orig,Subsidiary Legal Name_orig,Subsidiary Currency Code_orig,Subsidiary Is Active_orig,NetSuite Subsidiary ID_orig,VAT Registration Number,Is Active_orig,Is Elimination_orig,Subsidiary Region_orig,...,Subsidiary,%SubsidiaryCode,Subsidiary Legal Name,Subsidiary Currency Code,Subsidiary Is Active,NetSuite Subsidiary ID,Is Active,Is Elimination,Subsidiary Region,Workday Subsidiary Name
0,Qlik Foreign Parent AB - Do Not Use,NA1,Qlik Foreign Parent AB - Do Not Use,SEK,No,54,NaN,No,No,NaN,...,Hodges and Sons,7a1,Hodges and Sons,EUR,No,7a1,No,No,APAC,Hodges and Sons
1,Expressor Software Corporation,EXP,Expressor Software Corporation,USD,No,46,NaN,No,No,Americas,...,Peters Group,110,Peters Group,EUR,No,110,No,No,APAC,Peters Group
2,Purchase Price Adjustments,PPA,Purchase Price Adjustments,USD,Yes,56,NaN,Yes,No,Technologies,...,Russell LLC,c4f,Russell LLC,USD,Yes,c4f,Yes,No,APAC,Russell LLC
3,QlikTech Holdings Inc.,HOI,QlikTech Holdings Inc.,USD,Yes,13,NaN,Yes,No,Technologies,...,"Banks, Morales and Armstrong",e62,"Banks, Morales and Armstrong",SEK,Yes,e62,Yes,No,APAC,"Banks, Morales and Armstrong"
4,QlikTech Belgium,BEL,QlikTech Belgium,EUR,No,5,BE0848691897,No,No,EMEA,...,"Suarez, Johnson and Avery",b0a,"Suarez, Johnson and Avery",USD,No,b0a,No,No,World,"Suarez, Johnson and Avery"


## AR Comments

In [47]:
# read original file from gcs
df_comments = pd.read_csv(source_path+'AR_Comments.csv',nrows=100000)
df_comments[['comment_date','comment_text']]=df_comments['%ARCommentKey'].str.split('|',expand=True,n=1)
df_comments['comment_date']=pd.to_datetime(df_comments['comment_date']).dt.date
print('original dataframe')
display(df_comments.head())

# anonymize dataframe
anon_comments=dfAnonymizer(df_comments)
anon_comments.column_suppression(['comment_text'])
anon_comments.categorical_tokenization(['%ARCommentKey'],max_token_len=10,key=key)
anon_comments.datetime_noise('comment_date',seed=seed)

df_comments_anon=anon_comments.to_df()
df_comments_anon['AR Comments']=df_comments_anon['AR Comments'].apply(lambda x:0 if pd.isna(x) else 1)

anon_comments.info()

df_comments=df_comments.join(df_comments_anon,how='inner',lsuffix='_orig')
print('full dataframe')
display(df_comments.head())

#persist anonymized df to GCS
df_comments_anon.to_csv(destination_path+'anon_comments.csv',index=False)

# persist mapping tables to GCS
map_comment_key = dict(zip(df_comments['%ARCommentKey_orig'], df_comments['%ARCommentKey']))
with fs.open(destination_path+'map_comment_key.pickle', 'wb') as handle:
    pickle.dump(map_comment_key, handle, protocol=pickle.HIGHEST_PROTOCOL)

df_comments=df_comments_anon=[]


original dataframe


,%ARCommentKey,AR Comments,comment_date,comment_text
0,1/1/2019|230569 PT. Evotech Distribusi,NaN,2019-01-01,230569 PT. Evotech Distribusi
1,1/1/2019|230572 IDW2- IntegraÃ§Ã£o e Desenvolv...,12/31/18-VÃ­ctor- (renewal). End user informed...,2019-01-01,"230572 IDW2- IntegraÃ§Ã£o e Desenvolvimento, Lda."
2,1/1/2019|230578 BUSINESS & DECISION FRANCE,13/12 FM O/S in should be paid by EOM,2019-01-01,230578 BUSINESS & DECISION FRANCE
3,1/1/2019|230582 PT Mitra Integrasi Informatika,20/Aug/18 Vivien: Indomarco MA $1.8K - Invoice...,2019-01-01,230582 PT Mitra Integrasi Informatika
4,1/1/2019|230583 SSL Software Systems LLC,"21/12/2018 JIE//, renewal. Partner conf. wil...",2019-01-01,230583 SSL Software Systems LLC


+---------------+--------+-------------+-----------------------+
|    Column     | Status |    Type     |        Method         |
+===============+========+=============+=======================+
| %ARCommentKey | 1      | categorical | Tokenization          |
+---------------+--------+-------------+-----------------------+
| AR Comments   | 0      | categorical |                       |
+---------------+--------+-------------+-----------------------+
| comment_date  | 1      | categorical | Datetime Perturbation |
+---------------+--------+-------------+-----------------------+
| comment_text  | 1      | categorical | Column Suppression    |
+---------------+--------+-------------+-----------------------+
full dataframe


,%ARCommentKey_orig,AR Comments_orig,comment_date_orig,comment_text,%ARCommentKey,AR Comments,comment_date
0,1/1/2019|230569 PT. Evotech Distribusi,NaN,2019-01-01,230569 PT. Evotech Distribusi,84a2a45593,0,2019-08-31
1,1/1/2019|230572 IDW2- IntegraÃ§Ã£o e Desenvolv...,12/31/18-VÃ­ctor- (renewal). End user informed...,2019-01-01,"230572 IDW2- IntegraÃ§Ã£o e Desenvolvimento, Lda.",6303e07530,1,2019-03-02
2,1/1/2019|230578 BUSINESS & DECISION FRANCE,13/12 FM O/S in should be paid by EOM,2019-01-01,230578 BUSINESS & DECISION FRANCE,33b6ff8c43,1,2019-06-26
3,1/1/2019|230582 PT Mitra Integrasi Informatika,20/Aug/18 Vivien: Indomarco MA $1.8K - Invoice...,2019-01-01,230582 PT Mitra Integrasi Informatika,ba55a70d3f,1,2018-03-09
4,1/1/2019|230583 SSL Software Systems LLC,"21/12/2018 JIE//, renewal. Partner conf. wil...",2019-01-01,230583 SSL Software Systems LLC,08d451c05c,1,2018-12-27


## DSO

In [111]:
# read original file from gcs
df_dso = pd.read_csv(source_path+'AR_DSO.csv')
df_dso['NetSuite Extract DateTime']=pd.to_datetime(df_dso['NetSuite Extract DateTime'])
# split '%DSOKey' in period and subsidiary code to apply different anonymization
df_dso[['period','subsidiary_code']]=df_dso['%DSOKey'].str.split('|',expand=True)
df_dso['period']=pd.to_datetime(df_dso['period'])

print('original dataframe')
display(df_dso.head())

# read mapping tables from gcs

with fs.open(destination_path+'map_subsidiary_code.pickle', 'rb') as handle:
    map_subsidiary_code = pickle.load(handle)
with fs.open(destination_path+'map_subsidiary_currency_code.pickle', 'rb') as handle:
    map_subsidiary_currency_code = pickle.load(handle)

# anonymize dataframe
anon_dso=dfAnonymizer(df_dso)
anon_dso.datetime_noise(['NetSuite Extract DateTime','period'],seed=seed)
anon_dso.info()

df_dso_anon=anon_dso.to_df()
df_dso_anon['Transaction Line Amount - Local']=noise_amount_column(
    df_dso_anon['Transaction Line Amount - Local'])
df_dso_anon['Transaction Line Amount - USD']=noise_amount_column(
    df_dso_anon['Transaction Line Amount - USD'])
df_dso_anon['%DSOKey']=df_dso_anon[
    'period'].dt.strftime("%Y-%m")+'|'+df_dso_anon['subsidiary_code'].map(map_subsidiary_code)
df_dso_anon['From Currency Code']=df_dso_anon['From Currency Code'].map(map_subsidiary_currency_code)

# drop artificial columns created by splitting '%DSOKey'
df_dso=df_dso.drop(columns=['period','subsidiary_code'])
df_dso_anon=df_dso_anon.drop(columns=['period','subsidiary_code'])

# merge original and anonymized dataframes
df_dso=df_dso.join(df_dso_anon,how='inner',lsuffix='_orig')
print('full dataframe')
display(df_dso.head())

# persist anonymized df to GCS
df_dso_anon.to_csv(destination_path+'anon_dso.csv',index=False)

# persist mapping tables to GCS
map_dso_key = dict(zip(df_dso['%DSOKey_orig'], df_dso['%DSOKey']))
with fs.open(destination_path+'map_dso_key.pickle', 'wb') as handle:
    pickle.dump(map_dso_key, handle, protocol=pickle.HIGHEST_PROTOCOL)

df_dso=df_dso_anon=map_subsidiary_code=map_subsidiary_currency_code=[]

original dataframe


,NetSuite Extract DateTime,Transaction Line Amount - Local,Transaction Line Amount - USD,%DSOKey,DSO Amount Type,From Currency Code,period,subsidiary_code
0,2022-04-06 02:27:59,3912778.80,4.430557e+06,2022-01|FRA,Revenue,NaN,2022-01-01,FRA
1,2022-04-06 02:27:59,-15024.30,-1.682722e+04,2022-04|FRA,Revenue,NaN,2022-04-01,FRA
2,2022-04-06 02:27:59,3915833.96,4.309884e+06,2022-03|FRA,Revenue,NaN,2022-03-01,FRA
3,2022-04-06 02:27:59,2949581.23,3.346152e+06,2022-02|FRA,Revenue,NaN,2022-02-01,FRA
4,2022-04-06 02:27:59,2137978.60,3.162284e+05,2022-03|DMK,Revenue,NaN,2022-03-01,DMK


+---------------------------------+--------+-------------+-----------------------+
|             Column              | Status |    Type     |        Method         |
+=================================+========+=============+=======================+
| NetSuite Extract DateTime       | 1      | datetime    | Datetime Perturbation |
+---------------------------------+--------+-------------+-----------------------+
| Transaction Line Amount - Local | 0      | numeric     |                       |
+---------------------------------+--------+-------------+-----------------------+
| Transaction Line Amount - USD   | 0      | numeric     |                       |
+---------------------------------+--------+-------------+-----------------------+
| %DSOKey                         | 0      | categorical |                       |
+---------------------------------+--------+-------------+-----------------------+
| DSO Amount Type                 | 0      | categorical |                       |
+---

,NetSuite Extract DateTime_orig,Transaction Line Amount - Local_orig,Transaction Line Amount - USD_orig,%DSOKey_orig,DSO Amount Type_orig,From Currency Code_orig,NetSuite Extract DateTime,Transaction Line Amount - Local,Transaction Line Amount - USD,%DSOKey,DSO Amount Type,From Currency Code
0,2022-04-06 02:27:59,3912778.80,4.430557e+06,2022-01|FRA,Revenue,NaN,2022-11-29 02:27:59,2658519.2,3003704.5,2022-08|f11,Revenue,NaN
1,2022-04-06 02:27:59,-15024.30,-1.682722e+04,2022-04|FRA,Revenue,NaN,2022-06-08 02:27:59,-60016.2,-61218.1,2022-06|f11,Revenue,NaN
2,2022-04-06 02:27:59,3915833.96,4.309884e+06,2022-03|FRA,Revenue,NaN,2022-10-03 02:27:59,2660556.0,2923256.2,2022-08|f11,Revenue,NaN
3,2022-04-06 02:27:59,2949581.23,3.346152e+06,2022-02|FRA,Revenue,NaN,2021-06-06 02:27:59,2016387.5,2280768.3,2021-04|f11,Revenue,NaN
4,2022-04-06 02:27:59,2137978.60,3.162284e+05,2022-03|DMK,Revenue,NaN,2022-04-06 02:27:59,1475319.1,260818.9,2022-03|43f,Revenue,NaN


## Invoices

In [153]:
# read original file from gcs
df_invoice = pd.read_csv(source_path+'AR_Invoices.csv',nrows=10000)
date_columns=['Date','Due Date','As Of Date','Rev. Rec. Start Date','Rev. Rec. End Date','Contract Item Start Date','Contract Item End Date']
for column in date_columns:
    df_invoice[column]=pd.to_datetime(df_invoice[column])
df_invoice['Customer Code']=df_invoice['Customer Code'].astype(str)
df_invoice['PO Number']=df_invoice['PO Number'].astype(str)

# create 2 columns to store the original values of the columns to be anonymized
df_invoice[['document_id','item_id']]=df_invoice['%InvoiceItemKey'].str.split('|',expand=True,n=1)
print('original dataframe')
display(df_invoice.head())

# read mapping tables from gcs
with fs.open(destination_path+'map_subsidiary_currency_code.pickle', 'rb') as handle:
    map_subsidiary_currency_code = pickle.load(handle)
with fs.open(destination_path+'map_subsidiary_code.pickle', 'rb') as handle:
    map_subsidiary_code = pickle.load(handle)
with fs.open(destination_path+'map_comment_key.pickle', 'rb') as handle:
    map_comment_key = pickle.load(handle)
with fs.open(destination_path+'map_dso_key.pickle', 'rb') as handle:
    map_dso_key = pickle.load(handle)


# anonymize dataframe
anon_invoice=dfAnonymizer(df_invoice)
anon_invoice.column_suppression(['Detail URL','Customer URL','%SummaryKey','Project Name','Credit Limit'])
anon_invoice.datetime_noise(date_columns,seed=seed)
anon_invoice.categorical_fake(
    {'Customer Name':'company',
    'Sales Rep Name':'name',
    'Accounts Receivable Accountant':'name'},seed=seed)
anon_invoice.categorical_tokenization(['PO Number'],max_token_len=10,key=key)
anon_invoice.categorical_resampling(['Country Code'],seed=seed)
anon_invoice.info()

df_invoice_anon=anon_invoice.to_df()
value_columns=[
    'Temp Transaction Amount',
    'Temp Amount Due (Foreign Currency)',
    'Open Balance',
    'Amount Due (Foreign Currency)',
    'Transaction Amount',
    'Remaining (m)',
    'Recognized Balance',
    'Remaining Deferred Balance',
    'Tax Value',
    'Recognized Balance (Foreign Currency)',
    'Remaining Deferred Balance (Foreign Currency)',
    'Tax Value (Foreign Currency)',
    'Recognized Balance (Local)',
    'Remaining Deferred Balance (Local)',
    'Tax Value (Local)'
    ]
for column in value_columns:
    df_invoice_anon[column]=noise_amount_column(df_invoice_anon[column])

# anonymize %InvoiceItemKey and delete support fields
df_invoice_anon['document_id']=sequencial_values_for_column(df_invoice_anon['document_id'])
df_invoice_anon['%InvoiceItemKey']=df_invoice_anon['document_id'].astype(str)+'|'+df_invoice_anon['item_id'].astype(str)
df_invoice_anon=df_invoice_anon.drop(columns=['document_id','item_id'])
df_invoice=df_invoice.drop(columns=['document_id','item_id'])

# replace values with mapped values
df_invoice_anon['Transaction Currency']=df_invoice_anon['Transaction Currency'].map(map_subsidiary_currency_code)
df_invoice_anon['%ARCommentKey']=df_invoice_anon['%ARCommentKey'].map(map_comment_key)
df_invoice_anon['%DSOKey']=df_invoice_anon['%DSOKey'].map(map_dso_key)

# scramble values in 'Customer Code' and 'End User Code'
df_invoice_anon['Customer Code']=scramble_column(df_invoice_anon['Customer Code'].astype('str'))
df_invoice_anon['End User Code']=scramble_column(df_invoice_anon['End User Code'].astype('str'))
df_invoice_anon['Customer Original']=df_invoice_anon['Customer Code']+' '+df_invoice_anon['Customer Name']

# inherit values from other fields
df_invoice_anon['Customer']=df_invoice_anon['Customer Original']
df_invoice_anon['End User']=df_invoice_anon['End User Code']

df_invoice=df_invoice.join(df_invoice_anon,how='inner',lsuffix='_orig')

#persist anonymized df to GCS
df_invoice_anon.to_csv(destination_path+'anon_invoice.csv',index=False)

# persist mapping tables to GCS
map_invoice_key=dict(zip(df_invoice['%InvoiceItemKey_orig'], df_invoice['%InvoiceItemKey']))
with fs.open(destination_path+'map_invoice_key.pickle', 'wb') as handle:
    pickle.dump(map_invoice_key, handle, protocol=pickle.HIGHEST_PROTOCOL)

original dataframe


/var/folders/qq/4mccktj53_scgd15__4s1jvh0000gn/T/ipykernel_46833/3063599168.py:2: DtypeWarning: Columns (11,28,56,57,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df_invoice = pd.read_csv(source_path+'AR_Invoices.csv',nrows=10000)


,Is Missing Required PO,Detail URL,Customer URL,Collection Group,%SummaryKey,Document ID,%ItemID,%ARCommentKey,Due Date,Date,...,Country Code,Dedicated Account Rep,Is Dedicated Account,PO Required,Credit Status,Credit Limit,SFDC Account Record Type,%InvoiceItemKey,document_id,item_id
0,No,https://system.na1.netsuite.com/app/accounting...,https://system.na1.netsuite.com/app/common/ent...,Reseller,1/31/2019|230569 PT. Evotech Distribusi,INVSING00004586,2171.0,1/31/2019|230569 PT. Evotech Distribusi,2019-01-31,2019-01-01,...,ID,NaN,No,No,Approved,NaN,Partner Account,INVSING00004586|2171,INVSING00004586,2171
1,No,https://system.na1.netsuite.com/app/accounting...,https://system.na1.netsuite.com/app/common/ent...,Reseller,1/31/2019|230569 PT. Evotech Distribusi,INVSING00004587,2171.0,1/31/2019|230569 PT. Evotech Distribusi,2019-01-31,2019-01-01,...,ID,NaN,No,No,Approved,NaN,Partner Account,INVSING00004587|2171,INVSING00004587,2171
2,No,https://system.na1.netsuite.com/app/accounting...,https://system.na1.netsuite.com/app/common/ent...,Reseller,1/31/2019|230569 PT. Evotech Distribusi,INVSING00004620,2171.0,1/31/2019|230569 PT. Evotech Distribusi,2019-02-27,2019-01-28,...,ID,NaN,No,No,Approved,NaN,Partner Account,INVSING00004620|2171,INVSING00004620,2171
3,No,https://system.na1.netsuite.com/app/accounting...,https://system.na1.netsuite.com/app/common/ent...,Reseller,1/31/2019|230569 PT. Evotech Distribusi,INVSING00004620,10061.0,1/31/2019|230569 PT. Evotech Distribusi,2019-02-27,2019-01-28,...,ID,NaN,No,No,Approved,NaN,Partner Account,INVSING00004620|10061,INVSING00004620,10061
4,No,https://system.na1.netsuite.com/app/accounting...,https://system.na1.netsuite.com/app/common/ent...,Reseller,1/31/2019|230569 PT. Evotech Distribusi,INVSING00004620,2171.0,1/31/2019|230569 PT. Evotech Distribusi,2019-02-27,2019-01-28,...,ID,NaN,No,No,Approved,NaN,Partner Account,INVSING00004620|2171,INVSING00004620,2171


+-----------------------------------------------+--------+-------------+-----------------------+
|                    Column                     | Status |    Type     |        Method         |
+===============================================+========+=============+=======================+
| Is Missing Required PO                        | 0      | categorical |                       |
+-----------------------------------------------+--------+-------------+-----------------------+
| Detail URL                                    | 1      | categorical | Column Suppression    |
+-----------------------------------------------+--------+-------------+-----------------------+
| Customer URL                                  | 1      | categorical | Column Suppression    |
+-----------------------------------------------+--------+-------------+-----------------------+
| Collection Group                              | 0      | categorical |                       |
+-----------------------------

/var/folders/qq/4mccktj53_scgd15__4s1jvh0000gn/T/ipykernel_46833/2055437299.py:6: DeprecationWarning: The *random* parameter to shuffle() has been deprecated
since Python 3.9 and will be removed in a subsequent version.
  shuffle(scrambled_str,return_number)
